# Prep

In [1]:
import os
import random
import cv2
import numpy as np

grayscale = False

# Types: 142_class, 6_class, multi_model
dataset_type = '142_class'

image_dimensions = (512, 512)
data_subfolder_name = '../data'
use_dataset_percent = 0.5

# Legacy:
train_set_size = int(6199 * use_dataset_percent)
validation_set_size = int(1328 * use_dataset_percent)
test_set_size = int(1328 * use_dataset_percent)

# training_presets_subfolder_name = f'{dataset_type}/{str(image_dimensions[0])}_{str(int(use_dataset_percent * 100))}_{dataset_type}'
training_presets_subfolder_name = f'{dataset_type}/{str(image_dimensions[0])}'

print(training_presets_subfolder_name)
print(train_set_size, validation_set_size, test_set_size)

142_class/512
3099 664 664


In [24]:
%reset -f

# Dataset

### Info

- in general, there are 6 categories:
  - anchor(0)   - "long"
  - bolt(1)     - "long"
  - nut(2)      - "stubby"
  - screw(3)    - "long"
  - washer(4)   - "stubby"
  - other(5)    - "variable"

- balanced = the minimum amount of samples is in category #6, so we will adjust to it so we have an equal amount of samples in each category for now
- the training set will be 70% of the data, the validation and test will each be 15%
  - balanced = we use 8855 samples, which is only 52.18% of the dataset
  - x_unbalanced = x marks the percentage of the full dataset used

In [2]:
def count_files_in_folder(folder_path):
    file_count = 0
    
    for item in os.listdir(folder_path):
        if os.path.isfile(os.path.join(folder_path, item)):
            file_count += 1
    
    return file_count

folder_path = f'{data_subfolder_name}/0'  
num_files = count_files_in_folder(folder_path)
print("Number of files in the folder:", num_files)

folder_path = f'{data_subfolder_name}/1'  
num_files = count_files_in_folder(folder_path)
print("Number of files in the folder:", num_files)

folder_path = f'{data_subfolder_name}/2'  
num_files = count_files_in_folder(folder_path)
print("Number of files in the folder:", num_files)

folder_path = f'{data_subfolder_name}/3'  
num_files = count_files_in_folder(folder_path)
print("Number of files in the folder:", num_files)

folder_path = f'{data_subfolder_name}/4'  
num_files = count_files_in_folder(folder_path)
print("Number of files in the folder:", num_files)

folder_path = f'{data_subfolder_name}/5'  
num_files = count_files_in_folder(folder_path)
print("Number of files in the folder:", num_files)

Number of files in the folder: 1676
Number of files in the folder: 6078
Number of files in the folder: 1860
Number of files in the folder: 4079
Number of files in the folder: 1802
Number of files in the folder: 1476


### 142_class

##### Preparation

In [3]:
import datumaro as dm
import cv2
import json

dataset = dm.Dataset.import_from('../../fastener_dataset/annotations/instances_default.json', format='coco')
#stats = compute_ann_statistics(dataset)
#print(dataset)

# dataset_iter = iter(dataset)
dataset_list = list(dataset)

data = { }
 
for i in range(0, 16970):
    #print('processing image... ', i)
    item = dataset_list[i]
    item_annotation = item.annotations[0]
    
    new_path = item.media.path.replace(':', '_')
    # item_img = cv2.imread(new_path)
    # item_img = cv2.cvtColor(item_img, cv2.COLOR_BGR2RGB)
    # res = cv2.resize(item_img, dsize=(224, 224), interpolation=cv2.INTER_CUBIC)
    
    item_category = str(item_annotation.as_dict()['attributes']['category'])
    # print('annotation: ', item_category)
    
    # images.append(res)
    # labels.append(item_label)
    
    if(data.get(item_category) == None):
        data[item_category] = []
    data[item_category].append(new_path)

In [4]:
import json
import statistics

print(data.keys())

total_count = 0
adjusted_count = 0
adjusted_threshold = 100
counts = []
count_dict = {}

minimum = 17000
maximum = 0

for k, v in data.items():
    count_dict[k] = len(v)
    total_count += len(v)
    if len(v) < minimum:
        minimum = len(v)
    if len(v) > maximum:
        maximum = len(v)
    adjusted_count += min(len(v), adjusted_threshold)
    counts.append(len(v))

average_len = total_count / len(data.keys())
median_len = statistics.median(counts)

print(f'Minimum: {minimum}')
print(f'Maximum: {maximum}')
print(f'Average: {average_len}')
print(f'Median: {median_len}')
print(f'Total: {total_count}')
print(f'Adjusted total with threshold {adjusted_threshold}: {adjusted_count}')

# with open('./datasets/category_count.json', 'w') as file:
#     file.write(json.dumps(count_dict))

# Min samples for category: 12, Max samples for category: 582

dict_keys(['667', '626', '595', '581', '533', '491', '475', '441', '433', '427', '414', '403', '351', '348', '298', '290', '281', '278', '272', '250', '249', '223', '217', '211', '207', '183', '178', '177', '172', '168', '591', '534', '495', '492', '469', '380', '370', '305', '304', '302', '301', '300', '256', '253', '243', '242', '201', '193', '155', '144', '139', '132', '130', '126', '125', '119', '118', '115', '109', '108', '107', '106', '103', '668', '666', '665', '664', '663', '575', '538', '486', '483', '465', '460', '398', '397', '352', '329', '246', '245', '237', '233', '231', '224', '216', '215', '214', '213', '212', '209', '196', '195', '194', '142', '452', '381', '324', '323', '3221', '322', '317', '316', '3155', '315', '314', '3106', '310', '624', '421', '419', '413', '411', '405', '295', '294', '293', '270', '269', '268', '267', '265', '264', '262', '261', '260', '259', '258', '547', '392', '369', '368', '358', '357', '346', '335', '333', '232', '220', '190', '189', '147',

##### Create Single Set

In [4]:
# Parallel:
import concurrent.futures

def process_images(category, image_paths):
    processed_images = []
    for image_path in image_paths:
        image = cv2.imread(image_path)
        if grayscale is True:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            resized = cv2.resize(image, dsize=image_dimensions, interpolation=cv2.INTER_CUBIC)
            resized = np.expand_dims(resized, axis=-1)
        else:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            resized = cv2.resize(image, dsize=image_dimensions, interpolation=cv2.INTER_CUBIC)
        processed_images.append((resized, category))
    return processed_images

def create_dataset():
    dataset = []

    with concurrent.futures.ProcessPoolExecutor() as executor:
        futures = []
        for category in data.keys():
            random.shuffle(data[category])
            print(f'Category {category} with {len(data[category])} images...')
            futures.append(executor.submit(process_images, category, data[category][:len(data[category])//2]))

        for count, future in enumerate(concurrent.futures.as_completed(futures), 1):
            dataset.extend(future.result())
            print('\t', count, ' / ', len(data.keys()))

    random.shuffle(dataset)
    return dataset

dataset = create_dataset()
print('Done!')

Category 667 with 72 images...


Category 626 with 48 images...
Category 595 with 12 images...
Category 581 with 61 images...
Category 533 with 120 images...
Category 491 with 84 images...
Category 475 with 24 images...
Category 441 with 25 images...
Category 433 with 36 images...
Category 427 with 12 images...
Category 414 with 24 images...
Category 403 with 24 images...
Category 351 with 60 images...
Category 348 with 24 images...
Category 298 with 48 images...
Category 290 with 72 images...
Category 281 with 36 images...
Category 278 with 24 images...
Category 272 with 72 images...
Category 250 with 48 images...
Category 249 with 24 images...
Category 223 with 12 images...
Category 217 with 48 images...
Category 211 with 47 images...
Category 207 with 144 images...
Category 183 with 204 images...
Category 178 with 24 images...
Category 177 with 52 images...
Category 172 with 234 images...
Category 168 with 24 images...
Category 591 with 24 images...
Category 534 with 132 images...
Category 495 with 336 images...
Ca

In [5]:
import pickle

if grayscale is True:
    dataset_filpeath = f"./datasets/{training_presets_subfolder_name}/full_dataset_grayscale.pkl"
    with open(dataset_filpeath, "wb") as file:
        print('Saving dataset...')
        pickle.dump(dataset, file)
        print('Done!')
else:
    dataset_filpeath = f"./datasets/{training_presets_subfolder_name}/full_dataset.pkl"
    with open(dataset_filpeath, "wb") as file:
        print('Saving dataset...')
        pickle.dump(dataset, file)
        print('Done!')

Saving dataset...
Done!


##### Create 3 Sets

In [19]:
def create_dataset_slice(reduce_dataset: float = 1):
    train_data = []
    validation_data = []
    test_data = []
    
    count = 0
    for category in data.keys():
        
        random.shuffle(data[category])
        cutoff = int(len(data[category]))
        if int(len(data[category]) * reduce_dataset) > 100:
            # cutoff = int(len(data[category]) * reduce_dataset)
            cutoff = 100
        print(f'processing category {category} with {cutoff} / {len(data[category])} images...')
        data[category] = data[category][:cutoff+1]
        
        train_end = int(0.7 * len(data[category]))
        val_end = int(0.85 * len(data[category]))
        
        for image_path in data[category][:train_end]:
            image = cv2.imread(image_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            resized = cv2.resize(image, dsize=image_dimensions, interpolation=cv2.INTER_CUBIC)
            
            train_data.append((resized, category))
        for image_path in data[category][train_end:val_end]:
            image = cv2.imread(image_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            resized = cv2.resize(image, dsize=image_dimensions, interpolation=cv2.INTER_CUBIC)
            
            validation_data.append((resized, category))
        for image_path in data[category][val_end:]:
            image = cv2.imread(image_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            resized = cv2.resize(image, dsize=image_dimensions, interpolation=cv2.INTER_CUBIC)
            
            test_data.append((resized, category))
    
        count += 1
        print('\t', count, ' / ', len(data.keys()))
    
    random.shuffle(train_data)
    random.shuffle(validation_data)
    random.shuffle(test_data)
    
    return train_data, validation_data, test_data


train_set, validation_set, test_set = create_dataset_slice(use_dataset_percent)
print('Done!')

# (train_images, train_labels) = zip(*train_set)
# (validation_images, validation_labels) = zip(*validation_set)
# (test_images, test_labels) = zip(*test_set)

# train_images = np.array(train_images)
# train_labels = np.array(train_labels)
# validation_images = np.array(validation_images)
# validation_labels = np.array(validation_labels)
# test_images = np.array(test_images)
# test_labels = np.array(test_labels)

processing category 667 with 72 / 72 images...


	 1  /  142
processing category 626 with 48 / 48 images...
	 2  /  142
processing category 595 with 12 / 12 images...
	 3  /  142
processing category 581 with 61 / 61 images...
	 4  /  142
processing category 533 with 100 / 120 images...
	 5  /  142
processing category 491 with 84 / 84 images...
	 6  /  142
processing category 475 with 24 / 24 images...
	 7  /  142
processing category 441 with 25 / 25 images...
	 8  /  142
processing category 433 with 36 / 36 images...
	 9  /  142
processing category 427 with 12 / 12 images...
	 10  /  142
processing category 414 with 24 / 24 images...
	 11  /  142
processing category 403 with 24 / 24 images...
	 12  /  142
processing category 351 with 60 / 60 images...
	 13  /  142
processing category 348 with 24 / 24 images...
	 14  /  142
processing category 298 with 48 / 48 images...
	 15  /  142
processing category 290 with 72 / 72 images...
	 16  /  142
processing category 281 with 36 / 36 images...
	 17  /  142
processing category 278 with 24 / 

In [20]:
import pickle

training_presets_subfolder_name = f'{dataset_type}/{str(image_dimensions[0])}'

training_set_file_path = f"./datasets/{training_presets_subfolder_name}/training_set.pkl"
validation_set_file_path = f"./datasets/{training_presets_subfolder_name}/validation_set.pkl"
test_set_file_path = f"./datasets/{training_presets_subfolder_name}/test_set.pkl"

with open(training_set_file_path, "wb") as file:
    print('Saving training set...')
    pickle.dump(train_set, file)
    print('Done!')

with open(validation_set_file_path, "wb") as file:
    print('Saving validation set...')
    pickle.dump(validation_set, file)
    print('Done!')

with open(test_set_file_path, "wb") as file:
    print('Saving test set...')
    pickle.dump(test_set, file)
    print('Done!')

Saving training set...
Done!
Saving validation set...
Done!
Saving test set...
Done!


##### Experimental: Multithread

In [13]:
import concurrent.futures

def process_images(image_paths, category):
    processed_images = []
    for image_path in image_paths:
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        resized = cv2.resize(image, dsize=image_dimensions, interpolation=cv2.INTER_CUBIC)
        processed_images.append((resized, category))
    return processed_images

def create_dataset_slice(reduce_dataset: float = 1):
    train_data = []
    validation_data = []
    test_data = []

    count = 0
    futures = []

    with concurrent.futures.ThreadPoolExecutor() as executor:
        for category in data.keys():
            random.shuffle(data[category])
            cutoff = int(len(data[category]))
            if int(len(data[category]) * reduce_dataset) >= 25:
                cutoff = int(len(data[category]) * reduce_dataset)
            print(f'processing category {category} with {cutoff} / {len(data[category])} images...')
            data[category] = data[category][:cutoff]

            train_end = int(0.7 * len(data[category]))
            val_end = int(0.85 * len(data[category]))

            train_images = data[category][:train_end]
            val_images = data[category][train_end:val_end]
            test_images = data[category][val_end:]

            futures.append(executor.submit(process_images, train_images, category))
            futures.append(executor.submit(process_images, val_images, category))
            futures.append(executor.submit(process_images, test_images, category))

            count += 1
            print('\t', count, ' / ', len(data.keys()))

        for future in concurrent.futures.as_completed(futures):
            processed_images = future.result()
            if processed_images:
                if processed_images[0][1] in [category for category, _ in processed_images]:
                    train_data.extend(processed_images)
                elif processed_images[0][1] in [category for category, _ in processed_images]:
                    validation_data.extend(processed_images)
                else:
                    test_data.extend(processed_images)

    random.shuffle(train_data)
    random.shuffle(validation_data)
    random.shuffle(test_data)

    return train_data, validation_data, test_data

train_set, validation_set, test_set = create_dataset_slice(use_dataset_percent)
print('Done!')

# (train_images, train_labels) = zip(*train_set)
# (validation_images, validation_labels) = zip(*validation_set)
# (test_images, test_labels) = zip(*test_set)

# train_images = np.array(train_images)
# train_labels = np.array(train_labels)
# validation_images = np.array(validation_images)
# validation_labels = np.array(validation_labels)
# test_images = np.array(test_images)
# test_labels = np.array(test_labels)

processing category 667 with 72 / 72 images...
	 1  /  142
processing category 626 with 48 / 48 images...
	 2  /  142
processing category 595 with 12 / 12 images...
	 3  /  142
processing category 581 with 61 / 61 images...
	 4  /  142
processing category 533 with 30 / 120 images...
	 5  /  142
processing category 491 with 84 / 84 images...
	 6  /  142
processing category 475 with 24 / 24 images...
	 7  /  142
processing category 441 with 25 / 25 images...
	 8  /  142
processing category 433 with 36 / 36 images...
	 9  /  142
processing category 427 with 12 / 12 images...
	 10  /  142
processing category 414 with 24 / 24 images...
	 11  /  142
processing category 403 with 24 / 24 images...
	 12  /  142
processing category 351 with 60 / 60 images...
	 13  /  142
processing category 348 with 24 / 24 images...
	 14  /  142
processing category 298 with 48 / 48 images...
	 15  /  142
processing category 290 with 72 / 72 images...
	 16  /  142
processing category 281 with 36 / 36 images...
	

KeyboardInterrupt: 

##### Optional: Remove 'other' class

In [16]:
import pickle

with open(dataset_filpeath, "rb") as file:
  print('Loading dataset...')
  dataset = pickle.load(file)
  print('Done!')

Loading dataset...
Done!


In [17]:
import json

with open('datasets/categories.json', 'r') as file:
  categories = json.load(file)

dataset_no_other = []

print('Processing training set...')
for sample in dataset:
  if not int(sample[1]) in categories['5']:
    dataset_no_other.append(sample)
print('Done!')

    
print(len(dataset))
print(len(dataset_no_other))

Processing training set...
Done!
16970
15494


In [18]:
dataset_no_other_filepath = f"./datasets/{training_presets_subfolder_name}/full_dataset_NO_OTHER.pkl"

with open(dataset_no_other_filepath, "wb") as file:
    print('Saving dataset...')
    pickle.dump(dataset_no_other, file)
    print('Done!')


Saving dataset...
Done!


### 6_class

In [30]:
training_presets_subfolder_name = f'{dataset_type}/{str(image_dimensions[0])}_{str(int(use_dataset_percent * 100))}_{dataset_type}'

##### Create

In [3]:
from concurrent.futures import ThreadPoolExecutor

def process_class(i, count_train, count_validation, count_test):
    print('processing class... ', i)
    directory = f'{data_subfolder_name}/{str(i)}'
    files = os.listdir(directory)
    random.shuffle(files)

    train_files = files[:count_train]
    validation_files = files[count_train:count_train+count_validation]
    test_files = files[count_train+count_validation:count_train+count_validation+count_test]

    train_data = []
    validation_data = []
    test_data = []
    
    for file in train_files:
        new_path = f'{data_subfolder_name}/{str(i)}/{file.replace(":", "_")}'
        item_img = cv2.imread(new_path)
        if grayscale is True:
            item_img = cv2.cvtColor(item_img, cv2.COLOR_BGR2GRAY)
            res = cv2.resize(item_img, dsize=image_dimensions, interpolation=cv2.INTER_CUBIC)
            res = np.expand_dims(res, axis=-1)
        else:
            item_img = cv2.cvtColor(item_img, cv2.COLOR_BGR2RGB)
            res = cv2.resize(item_img, dsize=image_dimensions, interpolation=cv2.INTER_CUBIC)
        item_label = i
        train_data.append((res, item_label))
   
    print(f'\ttraining for {i} done...')
    for file in validation_files:
        new_path = f'{data_subfolder_name}/{str(i)}/{file.replace(":", "_")}'
        item_img = cv2.imread(new_path)
        if grayscale is True:
            item_img = cv2.cvtColor(item_img, cv2.COLOR_BGR2GRAY)
            res = cv2.resize(item_img, dsize=image_dimensions, interpolation=cv2.INTER_CUBIC)
            res = np.expand_dims(res, axis=-1)
        else:
            item_img = cv2.cvtColor(item_img, cv2.COLOR_BGR2RGB)
            res = cv2.resize(item_img, dsize=image_dimensions, interpolation=cv2.INTER_CUBIC)
        item_label = i
        validation_data.append((res, item_label))
        
    print(f'\tvalidation for {i} done...')
    for file in test_files:
        new_path = f'{data_subfolder_name}/{str(i)}/{file.replace(":", "_")}'
        item_img = cv2.imread(new_path)
        if grayscale is True:
            item_img = cv2.cvtColor(item_img, cv2.COLOR_BGR2GRAY)
            res = cv2.resize(item_img, dsize=image_dimensions, interpolation=cv2.INTER_CUBIC)
            res = np.expand_dims(res, axis=-1)
        else:
            item_img = cv2.cvtColor(item_img, cv2.COLOR_BGR2RGB)
            res = cv2.resize(item_img, dsize=image_dimensions, interpolation=cv2.INTER_CUBIC)
        item_label = i
        test_data.append((res, item_label))
    
    print(f'\ttest for {i} done...')
    return train_data, validation_data, test_data

def create_dataset_slice(count_train, count_validation, count_test):
    train_data = []
    validation_data = []
    test_data = []

    with ThreadPoolExecutor(max_workers=6) as executor:
        futures = [executor.submit(process_class, i, count_train, count_validation, count_test) for i in range(6)]
        
        for future in futures:
            t_data, v_data, te_data = future.result()
            train_data.extend(t_data)
            validation_data.extend(v_data)
            test_data.extend(te_data)

    random.shuffle(train_data)
    random.shuffle(validation_data)
    random.shuffle(test_data)

    return train_data, validation_data, test_data

train_set, validation_set, test_set = create_dataset_slice(train_set_size//6, validation_set_size//6, test_set_size//6)
print('Done!')

processing class...  0
processing class...  1
processing class...  2
processing class...  3
processing class...  4
processing class...  5


In [6]:
import pickle

training_set_file_path = f"./datasets/{training_presets_subfolder_name}/training_set.pkl"
validation_set_file_path = f"./datasets/{training_presets_subfolder_name}/validation_set.pkl"
test_set_file_path = f"./datasets/{training_presets_subfolder_name}/test_set.pkl"

with open(training_set_file_path, "wb") as file:
    print('Saving training set...')
    pickle.dump(train_set, file)
    print('Done!')

with open(validation_set_file_path, "wb") as file:
    print('Saving validation set...')
    pickle.dump(validation_set, file)
    print('Done!')

with open(test_set_file_path, "wb") as file:
    print('Saving test set...')
    pickle.dump(test_set, file)
    print('Done!')

Saving training set...
Done!
Saving validation set...
Done!
Saving test set...
Done!


##### Optional: Remove 'other' class

In [5]:
import pickle

training_set_file_path = f"./datasets/{training_presets_subfolder_name}/training_set.pkl"
validation_set_file_path = f"./datasets/{training_presets_subfolder_name}/validation_set.pkl"
test_set_file_path = f"./datasets/{training_presets_subfolder_name}/test_set.pkl"

with open(training_set_file_path, "rb") as file:
  print('Loading training set...')
  train_set = pickle.load(file)
  print('Done!')
  
with open(validation_set_file_path, "rb") as file:
  print('Loading validation set...')
  validation_set = pickle.load(file)
  print('Done!')
  
with open(test_set_file_path, "rb") as file:
  print('Loading test set...')
  test_set = pickle.load(file)
  print('Done!')

Loading training set...
Done!
Loading validation set...
Done!
Loading test set...
Done!


In [7]:
train_set_without_other = []
test_set_without_other = []
validation_set_without_other = []

print('Processing training set...')
for sample in train_set:
  if int(sample[1]) == 5:
    continue
  else:  
    train_set_without_other.append(sample)
print('Done!')
    
print('Processing test set...')
for sample in test_set:
  if int(sample[1]) == 5:
    continue
  else:  
    test_set_without_other.append(sample)
print('Done!')

print('Processing validation set...')
for sample in validation_set:
  if int(sample[1]) == 5:
    continue
  else:  
    validation_set_without_other.append(sample)    
print('Done!') 
  
print(len(train_set))
print(len(train_set_without_other))

print(len(validation_set))
print(len(validation_set_without_other))

print(len(test_set))
print(len(test_set_without_other))

Processing training set...
Done!
Processing test set...
Done!
Processing validation set...
Done!
3096
2580
660
550
660
550


In [8]:
training_set_file_path = f"./datasets/{training_presets_subfolder_name}/training_set_NO_OTHER.pkl"
validation_set_file_path = f"./datasets/{training_presets_subfolder_name}/validation_set_NO_OTHER.pkl"
test_set_file_path = f"./datasets/{training_presets_subfolder_name}/test_set_NO_OTHER.pkl"

with open(training_set_file_path, "wb") as file:
    print('Saving training set...')
    pickle.dump(train_set_without_other, file)
    print('Done!')

with open(validation_set_file_path, "wb") as file:
    print('Saving validation set...')
    pickle.dump(validation_set_without_other, file)
    print('Done!')

with open(test_set_file_path, "wb") as file:
    print('Saving test set...')
    pickle.dump(test_set_without_other, file)
    print('Done!')

Saving training set...
Done!
Saving validation set...
Done!
Saving test set...
Done!


##### Optional: Turn Grayscale

In [31]:
import pickle

training_set_file_path = f"./datasets/{training_presets_subfolder_name}/training_set.pkl"
validation_set_file_path = f"./datasets/{training_presets_subfolder_name}/validation_set.pkl"
test_set_file_path = f"./datasets/{training_presets_subfolder_name}/test_set.pkl"

with open(training_set_file_path, "rb") as file:
  print('Loading training set...')
  train_set = pickle.load(file)
  print('Done!')
  
with open(validation_set_file_path, "rb") as file:
  print('Loading validation set...')
  validation_set = pickle.load(file)
  print('Done!')
  
with open(test_set_file_path, "rb") as file:
  print('Loading test set...')
  test_set = pickle.load(file)
  print('Done!')

Loading training set...
Done!
Loading validation set...
Done!
Loading test set...
Done!


In [32]:
import cv2

train_set_grayscale = []
test_set_grayscale = []
validation_set_grayscale = []

print('Processing training set...')
for sample in train_set:
  item_img = sample[0]
  item_img = cv2.cvtColor(item_img, cv2.COLOR_BGR2GRAY)
  res = cv2.resize(item_img, dsize=image_dimensions, interpolation=cv2.INTER_CUBIC)
  res = np.expand_dims(res, axis=-1)
  train_set_grayscale.append((res, sample[1]))
print('Done!')
    
print('Processing test set...')
for sample in test_set:
  item_img = sample[0]
  item_img = cv2.cvtColor(item_img, cv2.COLOR_BGR2GRAY)
  res = cv2.resize(item_img, dsize=image_dimensions, interpolation=cv2.INTER_CUBIC)
  res = np.expand_dims(res, axis=-1)
  test_set_grayscale.append((res, sample[1]))
print('Done!')

print('Processing validation set...')
for sample in validation_set:
  item_img = sample[0]
  item_img = cv2.cvtColor(item_img, cv2.COLOR_BGR2GRAY)
  res = cv2.resize(item_img, dsize=image_dimensions, interpolation=cv2.INTER_CUBIC)
  res = np.expand_dims(res, axis=-1)
  validation_set_grayscale.append((res, sample[1]))
print('Done!') 

print(len(train_set_grayscale))
print(len(train_set))

Processing training set...
Done!
Processing test set...
Done!
Processing validation set...
Done!
3096
3096


In [33]:
training_set_file_path = f"./datasets/{training_presets_subfolder_name}/training_set_grayscale.pkl"
validation_set_file_path = f"./datasets/{training_presets_subfolder_name}/validation_set_grayscale.pkl"
test_set_file_path = f"./datasets/{training_presets_subfolder_name}/test_set_grayscale.pkl"

with open(training_set_file_path, "wb") as file:
    print('Saving training set...')
    pickle.dump(train_set_grayscale, file)
    print('Done!')

with open(validation_set_file_path, "wb") as file:
    print('Saving validation set...')
    pickle.dump(validation_set_grayscale, file)
    print('Done!')

with open(test_set_file_path, "wb") as file:
    print('Saving test set...')
    pickle.dump(test_set_grayscale, file)
    print('Done!')

Saving training set...
Done!
Saving validation set...
Done!
Saving test set...
Done!


# Additional Info

In [34]:
import pickle

training_set_file_path = f"./datasets/{training_presets_subfolder_name}/training_set.pkl"
validation_set_file_path = f"./datasets/{training_presets_subfolder_name}/validation_set.pkl"
test_set_file_path = f"./datasets/{training_presets_subfolder_name}/test_set.pkl"

with open(training_set_file_path, "rb") as file:
  print('Loading training set...')
  train_set = pickle.load(file)
  print('Done!')
  
with open(validation_set_file_path, "rb") as file:
  print('Loading validation set...')
  validation_set = pickle.load(file)
  print('Done!')
  
with open(test_set_file_path, "rb") as file:
  print('Loading test set...')
  test_set = pickle.load(file)
  print('Done!')

Loading training set...
Done!
Loading validation set...
Done!
Loading test set...
Done!


In [9]:
from sklearn.preprocessing import LabelEncoder
import json

all_labels = [label for _, label in train_set] + [label for _, label in validation_set] + [label for _, label in test_set]
all_labels = list(set(all_labels))

print(all_labels)
category_dict = {
  0: [],
  1: [],
  2: [],
  3: [],
  4: [],
  5: []
}
category_dict_reversed = {}

with open('../../fastener_dataset/annotations/instances_default.json', 'r') as file:
  annotations = json.load(file)
  annotations = annotations['annotations']
  for label in all_labels:
    for annotation in annotations:
      if int(annotation['attributes']['category']) == int(label):
        if category_dict_reversed.get(int(label)) is None:
          category_dict_reversed[int(label)] = []
        # print(f"Label: {label}, ID: {annotation['category_id']}")
        category_dict[int(annotation['category_id']) - 1].append(int(label))
        category_dict_reversed[int(label)].append(int(annotation['category_id']) - 1)
  
for key in category_dict_reversed.keys():
  category_dict_reversed[key] = list(set(category_dict_reversed[key]))
  if len(category_dict_reversed[key]) != 1:
    print(f'Warning!: {key} -> {category_dict_reversed[key]}')
  category_dict_reversed[key] = category_dict_reversed[key][0]
  
for key in category_dict.keys():
  category_dict[key] = list(set(category_dict[key]))
  print(f'{key} -> {len(category_dict[key])}')

with open('./datasets/categories.json', 'w') as file:
  file.write(json.dumps(category_dict))
  
with open('./datasets/categories_inverse.json', 'w') as file:
  file.write(json.dumps(category_dict_reversed))

['495', '212', '260', '3106', '290', '147', '196', '491', '207', '125', '329', '220', '215', '245', '348', '178', '223', '411', '300', '168', '233', '626', '441', '195', '242', '119', '324', '469', '155', '370', '315', '249', '333', '335', '3221', '139', '232', '115', '183', '106', '667', '209', '452', '666', '352', '486', '246', '298', '357', '294', '465', '368', '380', '547', '201', '268', '108', '665', '314', '189', '358', '591', '103', '269', '259', '369', '261', '293', '595', '109', '533', '323', '534', '664', '419', '144', '193', '581', '278', '624', '413', '3155', '132', '433', '272', '107', '492', '310', '190', '142', '256', '216', '253', '460', '305', '304', '398', '301', '265', '317', '414', '213', '177', '214', '392', '267', '302', '295', '663', '575', '264', '475', '403', '217', '322', '130', '137', '316', '427', '231', '118', '281', '237', '211', '346', '250', '172', '397', '224', '126', '668', '262', '270', '483', '421', '381', '243', '194', '538', '351', '258', '405']
0 